In [24]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import sys
import urllib.request

import pandas as pd 
import numpy as np 
from tqdm import tqdm

import dataload

import re
from bs4 import BeautifulSoup

import json
from pandas.io.json import json_normalize

In [26]:
# Data Load & Get Category
buy_full = dataload.load_buy()
print(buy_full.shape)

food = list(buy_full[buy_full["big_cat"]=="식품"].sm_cat.unique())
beauty = list(buy_full[buy_full["big_cat"]=="뷰티"].sm_cat.unique())
app = list(buy_full[buy_full["big_cat"]=="냉난방가전"].sm_cat.unique())
print("food : {} | beauty : {} | app : {}".format(len(food), len(beauty), len(app)))

(2056899, 6)
food : 212 | beauty : 131 | app : 40


In [45]:
buy_full.head(10)

,date,sex,age,big_cat,sm_cat,qty
0,20180101,F,20,식품,가공란,37
1,20180101,F,30,식품,가공란,16
2,20180101,F,40,식품,가공란,9
3,20180101,F,50,식품,가공란,3
4,20180101,M,20,식품,가공란,13
5,20180101,M,30,식품,가공란,6
6,20180101,M,40,식품,가공란,6
7,20180101,F,20,냉난방가전,가열식 가습기,1
8,20180101,F,40,냉난방가전,가열식 가습기,3
9,20180101,M,20,냉난방가전,가열식 가습기,1


# Naver Developers API 
* 참고 : [Parameters](https://developers.naver.com/docs/serviceapi/datalab/search/search.md#%EB%84%A4%EC%9D%B4%EB%B2%84-%ED%86%B5%ED%95%A9-%EA%B2%80%EC%83%89%EC%96%B4-%ED%8A%B8%EB%A0%8C%EB%93%9C-%EC%A1%B0%ED%9A%8C)
* 함정 : 비교 척도 **100** 기준 ㅠㅠ 

In [140]:
def keyword_search(keyword) : 
    ages = {20:['3','4'], 30:['5','6'], 40:['7','8'], 50:['9','10'], 60:['11']} ; genders = ["f", "m"]
    df = pd.DataFrame()
    
    for age, age_list in ages.items() : 
        for gender in genders : 
            client_id = "ZDHHjoLkaZSOVXsgJq_k"
            client_secret = "Arov4Zofoy"

            url = "https://openapi.naver.com/v1/datalab/search"
            body = {"startDate":"2018-01-01",
                    "endDate":"2019-12-31",
                    "timeUnit":"date",
                    "keywordGroups":[{"groupName":keyword,"keywords":[keyword]}],
                    "ages": age_list ,
                    "gender": gender}
            body_j = json.dumps(body)

            # get results 
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id",client_id)
            request.add_header("X-Naver-Client-Secret",client_secret)
            request.add_header("Content-Type","application/json")
            response = urllib.request.urlopen(request, data=body_j.encode("utf-8"))
            rescode = response.getcode()

            if(rescode==200):
                response_body = response.read()
                scraped = response_body.decode('utf-8')
                
                try : 
                    result = json_normalize(json.loads(scraped)["results"][0]["data"])
                    result.insert(1, "age", age) ; result.insert(1, "sex", gender.upper()) 
                    df = df.append(result, ignore_index=True)
                except : 
                    pass

            else:
                print("Error Code:" + rescode) 
    
    try : 
        df.insert(3, "sm_cat", keyword)
    except : 
        pass 
    
    return df

In [117]:
keyword_search("가공란")

,period,sex,age,sm_cat,ratio
0,2018-06-18,F,20,가공란,75.00000
1,2018-06-19,F,20,가공란,75.00000
2,2018-07-03,F,20,가공란,50.00000
3,2019-05-28,F,20,가공란,100.00000
4,2018-06-18,M,20,가공란,100.00000
5,2018-06-19,M,20,가공란,33.33333
6,2019-05-28,M,20,가공란,33.33333
7,2018-06-18,F,30,가공란,60.00000
8,2018-06-19,F,30,가공란,20.00000
9,2018-10-08,F,30,가공란,100.00000


In [177]:
def category_search(big_cat) : 
    df = pd.DataFrame()
    
    for sm_cat in tqdm(big_cat) : 
        tmp = keyword_search(sm_cat)
        print(sm_cat)
        print(tmp)
        df = df.append(tmp, ignore_index=True)
        
    if big_cat[0] in food : df.insert(3, "big_cat", "식품")
    elif big_cat[0] in beauty : df.insert(3, "big_cat", "뷰티")
    else : df.insert(3, "big_cat", "냉난방가전")
        
    return df 

In [178]:
food_search = category_search(food[:10])

 10%|█         | 1/10 [00:01<00:11,  1.28s/it]

가공란
        period sex  age sm_cat      ratio
0   2018-06-18   F   20    가공란   75.00000
1   2018-06-19   F   20    가공란   75.00000
2   2018-07-03   F   20    가공란   50.00000
3   2019-05-28   F   20    가공란  100.00000
4   2018-06-18   M   20    가공란  100.00000
5   2018-06-19   M   20    가공란   33.33333
6   2019-05-28   M   20    가공란   33.33333
7   2018-06-18   F   30    가공란   60.00000
8   2018-06-19   F   30    가공란   20.00000
9   2018-10-08   F   30    가공란  100.00000
10  2018-06-18   M   30    가공란  100.00000
11  2018-06-19   M   30    가공란   13.33333
12  2018-06-18   F   40    가공란  100.00000
13  2018-06-19   F   40    가공란   25.00000
14  2018-07-03   F   40    가공란   12.50000
15  2019-05-28   F   40    가공란   12.50000
16  2018-06-18   M   40    가공란  100.00000
17  2018-06-18   F   50    가공란  100.00000
18  2018-06-18   M   50    가공란  100.00000
19  2018-06-19   M   50    가공란   11.11111
20  2018-06-18   F   60    가공란  100.00000
21  2018-06-18   M   60    가공란  100.00000
22  2018-06-19   M   60    가공란

 20%|██        | 2/10 [00:03<00:12,  1.54s/it]

가자미
          period sex  age sm_cat    ratio
0     2018-01-01   F   20    가자미  0.96649
1     2018-01-02   F   20    가자미  0.70876
2     2018-01-03   F   20    가자미  1.03092
3     2018-01-04   F   20    가자미  1.14690
4     2018-01-05   F   20    가자미  0.97938
...          ...  ..  ...    ...      ...
7290  2019-12-27   M   60    가자미  3.53982
7291  2019-12-28   M   60    가자미  1.76991
7292  2019-12-29   M   60    가자미  1.47492
7293  2019-12-30   M   60    가자미  1.32743
7294  2019-12-31   M   60    가자미  2.65486

[7295 rows x 5 columns]


 30%|███       | 3/10 [00:04<00:10,  1.45s/it]

...

감마리놀렌산 영양제
        period sex  age      sm_cat      ratio
0   2018-01-09   F   20  감마리놀렌산 영양제   50.00000
1   2019-02-06   F   20  감마리놀렌산 영양제   33.33333
2   2019-04-23   F   20  감마리놀렌산 영양제   50.00000
3   2019-06-09   F   20  감마리놀렌산 영양제   33.33333
4   2019-08-18   F   20  감마리놀렌산 영양제   50.00000
5   2019-11-24   F   20  감마리놀렌산 영양제  100.00000
6   2019-11-26   F   20  감마리놀렌산 영양제   16.66666
7   2018-04-23   M   20  감마리놀렌산 영양제  100.00000
8   2019-02-01   F   30  감마리놀렌산 영양제  100.00000
9   2019-02-06   F   30  감마리놀렌산 영양제   60.00000
10  2019-04-23   F   30  감마리놀렌산 영양제   40.00000
11  2019-08-18   F   30  감마리놀렌산 영양제   40.00000
12  2019-11-10   F   30  감마리놀렌산 영양제   40.00000
13  2018-01-09   M   30  감마리놀렌산 영양제  100.00000
14  2018-04-23   M   30  감마리놀렌산 영양제  100.00000
15  2019-06-09   M   30  감마리놀렌산 영양제  100.00000
16  2018-01-09   F   40  감마리놀렌산 영양제   60.00000
17  2018-04-23   F   40  감마리놀렌산 영양제  100.00000
18  2019-02-19   F   40  감마리놀렌산 영양제   80.00000
19  2018-02-12   M   40  감마리놀렌산 영양제   28.57142
20

 90%|█████████ | 9/10 [00:14<00:01,  1.63s/it]

감말랭이
          period sex  age sm_cat     ratio
0     2018-01-01   F   20   감말랭이  66.66666
1     2018-01-02   F   20   감말랭이  80.12820
2     2018-01-03   F   20   감말랭이  86.53846
3     2018-01-04   F   20   감말랭이  87.82051
4     2018-01-05   F   20   감말랭이  72.43589
...          ...  ..  ...    ...       ...
7124  2019-12-27   M   60   감말랭이  42.42424
7125  2019-12-28   M   60   감말랭이  36.36363
7126  2019-12-29   M   60   감말랭이  62.12121
7127  2019-12-30   M   60   감말랭이  40.90909
7128  2019-12-31   M   60   감말랭이  42.42424

[7129 rows x 5 columns]


100%|██████████| 10/10 [00:16<00:00,  1.64s/it]

감자
          period sex  age sm_cat     ratio
0     2018-01-01   F   20     감자   3.42541
1     2018-01-02   F   20     감자   4.91712
2     2018-01-03   F   20     감자   6.38121
3     2018-01-04   F   20     감자   5.27624
4     2018-01-05   F   20     감자   4.39226
...          ...  ..  ...    ...       ...
7295  2019-12-27   M   60     감자  18.12080
7296  2019-12-28   M   60     감자  22.81879
7297  2019-12-29   M   60     감자  10.73825
7298  2019-12-30   M   60     감자  20.80536
7299  2019-12-31   M   60     감자  14.76510

[7300 rows x 5 columns]


In [179]:
food_search

,period,sex,age,big_cat,sm_cat,ratio
0,2018-06-18,F,20,식품,가공란,75.00000
1,2018-06-19,F,20,식품,가공란,75.00000
2,2018-07-03,F,20,식품,가공란,50.00000
3,2019-05-28,F,20,식품,가공란,100.00000
4,2018-06-18,M,20,식품,가공란,100.00000
...,...,...,...,...,...,...
29058,2019-12-27,M,60,식품,감자,18.12080
29059,2019-12-28,M,60,식품,감자,22.81879
29060,2019-12-29,M,60,식품,감자,10.73825
29061,2019-12-30,M,60,식품,감자,20.80536
